<a href="https://colab.research.google.com/github/NyusenKyusai/18018535_DataAnalytics/blob/main/TrainingWeakLinear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here I will creating a model with the dataset that is temperature, dew point, and has the strong outliers removed.

In [1]:
# needed to create the data frame
import pandas as pd

# create data frame from csv file we hosted on our github
df1 = pd.read_csv('https://raw.githubusercontent.com/NyusenKyusai/18018535_DataAnalytics/main/linearTDWeak.csv', index_col=0, )
df2 = pd.read_csv('https://raw.githubusercontent.com/NyusenKyusai/18018535_DataAnalytics/main/linearTDSnowWeak.csv', index_col=0, )
df3 = pd.read_csv('https://raw.githubusercontent.com/NyusenKyusai/18018535_DataAnalytics/main/linearTDMaxMinWeak.csv', index_col=0, )
df4 = pd.read_csv('https://raw.githubusercontent.com/NyusenKyusai/18018535_DataAnalytics/main/linearTDMaxMinSnowWeak.csv', index_col=0, )

In [2]:
# make sure we have our data by printing it out
print(df1[:6])

    temp  dewp  NUM_CRIMES
2   25.7  16.0         674
3   31.5  27.0         648
4   29.0  26.1         514
5    2.2  -7.4         523
9    8.1  -0.5         590
10   4.8  -3.9         647


In [166]:
# needed to help with speedy maths based calculations
import numpy as np

# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df4.iloc[np.random.permutation(len(df4))]

# d1 = 2
# d2 = 3
# d3 = 4
# d4 = 5
# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:5]

# print out the first 6 rows of predictors.
print(predictors[:6])

      temp  dewp   max   min  sndp
128   71.1  60.7  82.9  52.0   0.0
572   79.0  66.4  93.0  71.1   0.0
1800  35.7  27.9  45.0  27.0   0.0
691   30.1  17.2  41.0  21.9   0.0
1716  71.9  66.8  89.1  63.0   0.0
1473  30.2  23.8  32.0  17.1   2.0


In [167]:
# print out the shuffled data (first 5 rows)
shuffle[:5]

,temp,dewp,max,min,sndp,NUM_CRIMES
128,71.1,60.7,82.9,52.0,0.0,807
572,79.0,66.4,93.0,71.1,0.0,827
1800,35.7,27.9,45.0,27.0,0.0,700
691,30.1,17.2,41.0,21.9,0.0,683
1716,71.9,66.8,89.1,63.0,0.0,784


In [168]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

128     807
572     827
1800    700
691     683
1716    784
1473    673
Name: NUM_CRIMES, dtype: int64


In [169]:
# A scale is not required here, but the constant will be useful in the assignment.
SCALE_NUM_TRIPS = 1.0

In [170]:
# Split our data into a training set i.e. 75% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_CRIMES'])*0.75)
# Split our data into a test set i.e. 20% of the length of the shuffle array
testsize = int(len(shuffle['NUM_CRIMES'])*0.2)
# The final test set size is 100% - 75% - 20% = 5% of the length of the shuffle array.
finaltestsize = len(shuffle['NUM_CRIMES']) - trainsize - testsize

# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

In [171]:
# import tensorflow
%tensorflow_version 1.x
import tensorflow as tf

# check the version
print(tf.__version__)

# needed for high-level file management
import shutil  

# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
# linearTDWeak
# linearTDSnowWeak
# linearTDMaxMinWeak
# linearTDMaxMinSnowWeak
shutil.rmtree('/tmp/linearTDMaxMinSnowWeak5', ignore_errors=True)

# This is the core of our linear regressor

# You can see that we save the model, use the the Adam optimization algorithm, which is an extension 
# to stochastic gradient descent that has recently seen broader adoption for deep learning applications 
# in computer vision and natural language processing and infer real valued columns from input which interprets 
# all inputs as dense, fixed-length float values.

# See the link for more information
# https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/contrib/learn/LinearRegressor
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linearTDMaxMinSnowWeak5', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_TRIPS, steps=10000)

# Next, we can check our predictions based on our predictors.
# It goes from trainsize to testsize + trainsize to keep the last 5% out of the model
# We can test with the last 5% later
preds = estimator.predict(x=predictors[trainsize:testsize + trainsize].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_NUM_TRIPS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:testsize + trainsize].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Number of Crimes Values.
avg = np.mean(shuffle['NUM_CRIMES'][:trainsize])

# Calculate the RMSE using Number of Crimes Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_CRIMES'][trainsize:testsize + trainsize] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

1.15.2
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff487fae690>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linearTDMaxMinSnowWeak5', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running lo

Temp Dewp  
Model 1 got a RMSE of 53.1879 | Using the average of 730.4246 it got an RMSE of 77.0098  
Model 2 got a RMSE of 53.3371 | Using the average of 731.2799 it got an RMSE of 78.1301  
Model 3 got a RMSE of 58.3367 | Using the average of 728.2545 it got an RMSE of 78.9838  
Model 4 got a RMSE of 51.9004 | Using the average of 730.0007 it got an RMSE of 75.7996  
Model 5 got an RMSE of 53.7021 | Using the average of 729.8448 it got an RMSE of 75.7200  

Temp Dewp Snow  
Model 1 got a RMSE of 54.2052 | Using the average of 731.0612 it got an RMSE of 78.2361  
Model 2 got a RMSE of 52.2373 | Using the average of 729.8478 it got an RMSE of 77.9094  
Model 3 got a RMSE of 53.9293 | Using the average of 731.5626 it got an RMSE of 78.2305  
Model 4 got a RMSE of 54.0279 | Using the average of 729.2828 it got an RMSE of 75.6122  
Model 5 got an RMSE of 54.1470 | Using the average of 731.2231 it got an RMSE of 79.2849  


Temp Dewp Max Min  
Model 1 got a RMSE of 55.0659 | Using the average of 729.0015 it got an RMSE of 75.2594  
Model 2 got a RMSE of 53.4855 | Using the average of 730.5134 it got an RMSE of 76.8938  
Model 3 got a RMSE of 53.6011 | Using the average of 727.4918 it got an RMSE of 77.0739  
Model 4 got a RMSE of 53.8609 | Using the average of 729.1358 it got an RMSE of 77.6193  
Model 5 got an RMSE of 53.5432 | Using the average of 729.7963 it got an RMSE of 78.5241   

Temp Dew Max Min Sndp  
Model 1 got a RMSE of 50.2573 | Using the average of 729.8657 it got an RMSE of 77.3670  
Model 2 got a RMSE of 52.5694 | Using the average of 730.8866 it got an RMSE of 76.3152  
Model 3 got a RMSE of 51.9747 | Using the average of 730.8522 it got an RMSE of 75.6021  
Model 4 got a RMSE of 50.7424 | Using the average of 731.3545 it got an RMSE of 74.4127  
Model 5 got an RMSE of 52.4102 | Using the average of 730.4425 it got an RMSE of 78.4425   

In [172]:
# Creating a variable for the last 5% of the data that is easy to handle
input = predictors[testsize + trainsize:]
					
# Boilerplate for creating the estimator
# linearTDWeak
# linearTDSnowWeak
# linearTDMaxMinWeak
# linearTDMaxMinSnowWeak
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linearTDMaxMinSnowWeak5', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

# Predicting the values 
preds = estimator.predict(x=input.values)

# Creating a list of predictions 
predslist = preds['scores']*SCALE_NUM_TRIPS

# Creating the RMSE by comparing the targets to the list of predictions
rmse = np.sqrt(np.mean((targets[testsize + trainsize:].values - predslist)**2))

# Calculate the mean of the Number of Crimes Values.
avg = np.mean(shuffle['NUM_CRIMES'][testsize + trainsize:])

corr_matrix = np.corrcoef(targets[testsize + trainsize:].values, predslist)
corr = corr_matrix[0,1]
R_sq = corr**2

print()
# Printing the rmse and average crimes
print('LinearRegression has RMSE of {0}'.format(rmse));
print('Average is {0}'.format(avg))
print('The coefficient of determination is {0}'.format(R_sq))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff4881afa90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linearTDMaxMinSnowWeak5', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linearTDMaxMinSnowWeak5/model.ckpt-10000
INFO:te

Temp Dewp  
Model 1 got a RMSE of 53.0432 and a r^2 of 0.4924  
Model 2 got a RMSE of 53.3792 and a r^2 of 0.5344  
Model 3 got a RMSE of 54.9113 and a r^2 of 0.5161  
Model 4 got a RMSE of 55.3390 and a r^2 of 0.5491  
Model 5 got an RMSE of 53.5846 and a r^2 of 0.5834  

Temp Dewp Sndp  
Model 1 got a RMSE of 46.5864 and a r^2 of 0.6221  
Model 2 got a RMSE of 54.9028 and a r^2 of 0.5367  
Model 3 got a RMSE of 50.8214 and a r^2 of 0.6143  
Model 4 got a RMSE of 62.2075 and a r^2 of 0.4291  
Model 5 got an RMSE of 42.5731 and a r^2 of 0.6129  

Temp Dewp Max Min  
Model 1 got a RMSE of 53.9638 and a r^2 of 0.4492  
Model 2 got a RMSE of 50.2790 and a r^2 of 0.6400  
Model 3 got a RMSE of 59.4078 and a r^2 of 0.4375  
Model 4 got a RMSE of 59.3535 and a r^2 of 0.5510  
Model 5 got an RMSE of 53.6890 and a r^2 of 0.5399  


Temp Dewp Max Min Sndp  
Model 1 got a RMSE of 56.1904 and a r^2 of 0.5295  
Model 2 got a RMSE of 51.4885 and a r^2 of 0.5613  
Model 3 got a RMSE of 60.2221 and a r^2 of 0.4788  
Model 4 got a RMSE of 50.5584 and a r^2 of 0.5483  
Model 5 got an RMSE of 54.6416 and a r^2 of 0.4236  